In [ ]:
"""
activity i need to capture:

- total commits
- total lines of code added
- total lines of code removed

- commits per repository 
    - display org and reponame together with number of commits
- repositories created
    - name of repo
- issues opened 
    - name of repo and headline of issue
- pull requests opened
    - name of repo and title of pull request


# how it works

- github cronjob daily at the end of my day 00:00 - collects data until back 00:01
- telegram bot sends me a message with the data formatted
"""

In [30]:
import requests
from datetime import datetime, timedelta
import pytz
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def fetch_last_event_local_time(username, token):
    if not username or not token:
        raise ValueError("GitHub username or token is not set.")
    
    headers = {'Authorization': f'token {token}'}
    url = f'https://api.github.com/users/{username}/events'
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching events: {response.status_code}")

    events = response.json()
    if not events:
        print("No events found for the user.")
        return None
    
    last_event_time_utc = events[0]['created_at']  # The last event's timestamp

    # Convert to local time
    dt = datetime.strptime(last_event_time_utc, '%Y-%m-%dT%H:%M:%SZ')
    utc_time = dt.replace(tzinfo=pytz.UTC)
    local_time = utc_time.astimezone()  # Convert to local timezone

    return local_time

In [31]:
def get_time_24_hours_back(event_time):
    # Get the current time in the same timezone as the event time
    current_time = datetime.now(event_time.tzinfo)
    
    # Calculate the time 24 hours back
    time_24_hours_back = current_time - timedelta(hours=24)
    
    return time_24_hours_back

Current time: 2024-09-07 13:09:56.142089+02:00
Time 24 hours back: 2024-09-06 13:09:56.142084+02:00


In [42]:
def fetch_user_events(username, token):
    if not username or not token:
        raise ValueError("GitHub username or token is not set.")
    
    headers = {
        'Accept': 'application/vnd.github+json',
        'Authorization': f'Bearer {token}',
        'X-GitHub-Api-Version': '2022-11-28'
    }
    url = f'https://api.github.com/users/{username}/events'
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching events: {response.status_code}")

    events = response.json()
    return events

def filter_commits_last_24_hours(events, since, until):
    commits = []
    for event in events:
        if event['type'] == 'PushEvent':
            event_time = datetime.strptime(event['created_at'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC)
            if since <= event_time <= until:
                for commit in event['payload']['commits']:
                    commits.append(commit)
    return commits

# Example usage
username = os.getenv('GITHUB_USER')
token = os.getenv('GITHUB_TOKEN')

# Step 1: Get the last event time in local timezone
event_time = fetch_last_event_local_time(username, token)
if event_time:
    # Step 2: Calculate the time 24 hours back
    time_24_hours_back = get_time_24_hours_back(event_time)
    
    # Step 3: Fetch all recent events
    events = fetch_user_events(username, token)
    
    if events:
        # Step 4: Filter commits from the last 24 hours
        commits_last_24_hours = filter_commits_last_24_hours(events, time_24_hours_back, event_time)
        print("Commits in the last 24 hours:", commits_last_24_hours)
        print("Total commits in the last 24 hours:", len(commits_last_24_hours))

Commits in the last 24 hours: [{'sha': '2214f255f64d6740b59f915e5ee08579f0090eac', 'author': {'email': '68147541+yachty66@users.noreply.github.com', 'name': 'Max Hager'}, 'message': 'Add GitHub activity tracking functionality to main notebook, including fetching user events and displaying commits in the last 24 hours. Updated execution count and created test data file.', 'distinct': True, 'url': 'https://api.github.com/repos/yachty66/ai_code_reviewer/commits/2214f255f64d6740b59f915e5ee08579f0090eac'}, {'sha': '2f215a123fd93134818ffae7377ee79d523a805e', 'author': {'email': '68147541+yachty66@users.noreply.github.com', 'name': 'Max Hager'}, 'message': 'test', 'distinct': True, 'url': 'https://api.github.com/repos/yachty66/ai_code_reviewer/commits/2f215a123fd93134818ffae7377ee79d523a805e'}, {'sha': '1af1355599060a088cf9359ce942a5abf5fd2b1d', 'author': {'email': '68147541+yachty66@users.noreply.github.com', 'name': 'Max Hager'}, 'message': 'Add GitHub activity tracking functionality to mai